In [4]:
from tools import open_json


In [5]:
data = open_json('2023_season.json')
print(len(data))

285


In [6]:
game1 = data['20230911_BUF@NYJ']
print(game1.keys())

dict_keys(['statusCode', 'body'])


In [10]:
print(game1['body']['playerStats'].keys())
print(game1['body']['playerStats']['2577466'])

dict_keys(['4039007', '14053', '4035004', '3043136', '3120464', '4240090', '3125232', '4037584', '3915486', '11260', '2581273', '3122168', '3895791', '3125114', '3916577', '2976549', '3917660', '3930086', '4033748', '2977679', '3046287', '4569618', '4240456', '4240896', '4240532', '4427366', '3051324', '3117258', '4240657', '4428696', '3042717', '16910', '3895798', '16759', '4411193', '3121003', '4361767', '2582132', '13493', '2983155', '3128390', '3929956', '4240585', '4427250', '4567962', '3858271', '4243537', '2577466', '4039375', '3929631', '16720', '4379399', '2970625', '4037521', '4039303', '2512477', '15960', '4361259', '2577078', '3918298', '3139387', '4361331', '4259646', '4030899', '2976249', '4039505', '4361372', '15928', '3057524', '3110565', '4259166', '3925347', '15920', '4040982', '14993', '4568624', '2577637', '4035232', '4385690', '3116593', '3917232', '2976212', '2574576', '4428681', '4427278', '4373678', '8439', '2577367', '2514122', '4379778', '4362506', '15979', '4

In [7]:
#for all games, we want log the stats of each participating player.

all_players = {}


for game_id in data.keys():
    player_stats = data[game_id]['body']['playerStats']
    for player_id, player_stat in player_stats.items():
        if player_id not in all_players: all_players[player_id] = []
        all_players[player_id].append(player_stat)


In [8]:

print(len(all_players))


2145


In [28]:
#let's get the total tackles across a whole season of a specific player

taylor = all_players['4039007']
season_tackles = [taylor[i]['Defense']['totalTackles'] for i in range(len(taylor)) if 'Defense' in taylor[i]]
print(season_tackles)

['5', '1', '1', '6', '1', '1', '3', '7', '3', '3', '8', '9', '1', '1']


In [9]:
all_players_clean = {}

# let's try separating players by position
for player_id, player_stat in all_players.items():
    if len(player_stat) > 6:
        all_players_clean[player_id] = player_stat

In [49]:
#print(len(all_players_clean))
import numpy as np

#get average size of player stats
#print(np.mean(np.array([len(player_stat) for player_stat in all_players_clean.values()])))


player_names = [player[0]['longName'] for player in all_players_clean.values()]

In [50]:
print(len(player_names))
print(len(set(player_names)))
print(len(all_players_clean.items()))

1674
1669
1674


In [45]:
import random
rand_player = random.randint(0,len(all_players_clean))
rand_id = list(all_players_clean.keys())[rand_player]
print(all_players_clean[rand_id][0])
print(all_players_clean[rand_id][0]['longName'])
print(all_players_clean[rand_id][0]['playerID'])

{'gameID': '20240115_PIT@BUF', 'teamID': '26', 'snapCounts': {'offSnapPct': '0.00', 'defSnap': '49', 'stSnap': '8', 'stSnapPct': '0.30', 'offSnap': '0', 'defSnapPct': '0.73'}, 'team': 'PIT', 'teamAbv': 'PIT', 'Defense': {'totalTackles': '2', 'defTD': '0', 'forcedFumbles': '0', 'soloTackles': '2', 'tfl': '0', 'qbHits': '0', 'defensiveInterceptions': '0', 'sacks': '0', 'passDeflections': '0'}, 'playerID': '4426506', 'longName': 'Joey Porter Jr.'}
Joey Porter Jr.
4426506


In [48]:
from player import Player

def record_player_season_stats(player_id: str, all_players_dict: dict) -> Player:
    assert player_id in all_players_dict.keys() and len(list(all_players_dict[player_id])) >= 8

    player_name = all_players_dict[player_id][0]['longName']
    print(player_name)
    player = Player(player_id, player_name)
    
    entire_season = all_players_dict[player_id]

    for game in entire_season:
        player.total_games_recorded += 1
        if 'snapCounts' in game:
            sc = game['snapCounts']
            player.total_offensive_snapPct.append(float(sc['offSnapPct']))
            player.total_offensive_snaps.append(float(sc['offSnap']))

            player.total_defensive_snapPct.append(float(sc['defSnapPct']))
            player.total_defensive_snaps.append(float(sc['defSnap']))

            player.total_st_snapPct.append(float(sc['stSnapPct']))
            player.total_st_snaps.append(float(sc['stSnap']))

        if 'Rushing' in game:
            rs = game['Rushing']
            player.total_rushAvg.append(float(rs['rushAvg']))
            player.total_rushYds.append(float(rs['rushYds']))
            player.total_carries.append(float(rs['carries']))
            player.total_rushTD.append(float(rs['rushTD']))

        if 'Receiving' in game:
            rc = game['Receiving']
            player.total_receptions.append(float(rc['receptions']))
            player.total_recYds.append(float(rc['recYds']))
            player.total_recAvg.append(float(rc['recAvg']))
            player.total_targets.append(float(rc['targets']))

        if 'Passing' in game:
            ps = game['Passing']
            player.total_qbr.append(float(ps['qbr']))
            player.total_rtg.append(float(ps['rtg']))
            player.total_passAttempts.append(float(ps['passAttempts']))
            player.total_passAvg.append(float(ps['passAvg']))
            player.total_passTD.append(float(ps['passTD']))
            player.total_passYds.append(float(ps['passYds']))
            player.total_int.append(float(ps['int']))
            player.total_passCompletions.append(float(ps['passCompletions']))

        if 'Defense' in game:
            df = game['Defense']
            print(df)
            player.total_totalTackles.append(float(df['totalTackles']))
            player.total_soloTackles.append(float(df['soloTackles']))
            player.total_forcedFumbles.append(float(df['forcedFumbles']))
            player.total_sacks.append(float(df['sacks']))
            player.total_defInterceptions.append(float(df['defensiveInterceptions']))
            player.total_passDeflections.append(float(df['passDeflections']))


pid = '4430807'
p = record_player_season_stats(pid, all_players_clean)

Bijan Robinson
{'fumblesLost': '1', 'defensiveInterceptions': '0', 'forcedFumbles': '0', 'fumbles': '1', 'fumblesRecovered': '0'}


KeyError: 'totalTackles'

In [10]:
from tools import save_json

save_json('all_players_clean_2023.json',all_players_clean)

Data has been saved to all_players_clean_2023.json
